In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

### Installing beautifulsoup4 to explore the input data

In [7]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.7.1       |           py36_1         143 KB  anaconda
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    certifi-2019.6.16          |           py36_0         154 KB  anaconda
    soupsieve-1.8              |           py36_0         104 KB  anaconda
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.8-py36_0        a

In [8]:
from bs4 import BeautifulSoup

### Download and parse the data

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [9]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [10]:
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

In [11]:
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

'''for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    print(row)  '''
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

toronto_data = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"]).replace('\n','', regex=True)
toronto_data=toronto_data.drop(toronto_data.index[[0]])
toronto_data.head()
#toronto_data.sort_values(["PostalCode"], axis=0,ascending=True, inplace=True)

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### 288 rows fetched

In [12]:
toronto_data.shape

(288, 3)

### Ignore cells with a borough that is Not assigned.

In [13]:
toronto_data.drop(toronto_data[toronto_data['Borough'] == "Not assigned"].index, inplace = True)
toronto_data.shape

(211, 3)

### These two rows will be combined into one row with the neighborhoods separated with a comma

In [14]:
toronto_data_merge = toronto_data.groupby(['PostalCode','Borough'],as_index=False,sort=False).agg(','.join)

In [15]:
toronto_data_merge.head(25)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### filter rows having  Neighborhood as "Not assigned" and replace as like will column borough #ex:PostalCode M7A

In [105]:
#toronto_data_merge_del = toronto_data_merge[toronto_data_merge['Neighborhood']=='Not assigned']
#toronto_data_merge_del.shape #only record M7A	Queen's Park	Not assigned
#toronto_data_merge[toronto_data_merge['PostalCode']=='M7A']

In [16]:
toronto_data_merge.loc[toronto_data_merge['Neighborhood']=='Not assigned', 'Neighborhood'] = toronto_data_merge['Borough']
#toronto_data_merge[toronto_data_merge['PostalCode']=='M7A']
toronto_data_merge #ex:PostalCode M7A

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


### use the .shape method to print the number of rows of your dataframe.

In [17]:
toronto_data_merge.shape

(103, 3)

In [2]:
conda install -c conda-forge geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

### Importing geocode data from http://cocl.us/Geospatial_data and creating Latitude and Longitude column

In [18]:
filename = "http://cocl.us/Geospatial_data"
headers = ["PostalCode","Latitude","Longitude"]
Geospatial_data = pd.read_csv(filename,names = headers)
#Geospatial_data = pd.read_csv(filename)
Geospatial_data=Geospatial_data.drop(Geospatial_data.index[[0]])
Geospatial_data.head()


,PostalCode,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174
5,M1H,43.773136,-79.2394761


In [19]:
#toronto_data_merge.sort_values(["PostalCode"], axis=0,ascending=True, inplace=True)
toronto_data_merge.shape #(103, 3)
Geospatial_data.shape #(103, 3)


(103, 3)

In [20]:
neighborhoods = pd.merge(toronto_data_merge, Geospatial_data, on ='PostalCode')
neighborhoods.shape
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
5,M9A,Etobicoke,Islington Avenue,43.6678556,-79.5322424
6,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534
7,M3B,North York,Don Mills North,43.7459058,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6571618,-79.3789371


### Use geopy library to get the latitude and longitude values of Toronto City

In [21]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [29]:
neighborhoods.Latitude = neighborhoods.Latitude.astype(float)
neighborhoods.Longitude = neighborhoods.Longitude.astype(float)
neighborhoods.dtypes


PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

### Get unique values in column Borough

In [31]:
neighborhoods.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

#### Create a map of Toronto with neighborhoods superimposed on top.

In [30]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in "Central Toronto". So let's slice the original dataframe and create a new dataframe of the "Central Toronto" data.

In [44]:
CentralToronto_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
CentralToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


Let's get the geographical coordinates of Central Toronto.

In [45]:
address = 'Central Toronto, Toronto canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


let's visualizat Central Toronto the neighborhoods in it.

In [46]:
# create map of Manhattan using latitude and longitude values
map_CentralToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(CentralToronto_data['Latitude'], CentralToronto_data['Longitude'], CentralToronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CentralToronto)  
    
map_CentralToronto

#### Define Foursquare Credentials and Version

In [47]:
CLIENT_ID = 'OZNSZDTSIOL1JOGR2BOJMLQBSCV1GQEYQFU0C1XWW3DHHAHQ' # your Foursquare ID
CLIENT_SECRET = 'C5DINZ0AKSXVYAIWQUKRUI40LHTLZE2VE2YOC4BY13JK1B55' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OZNSZDTSIOL1JOGR2BOJMLQBSCV1GQEYQFU0C1XWW3DHHAHQ
CLIENT_SECRET:C5DINZ0AKSXVYAIWQUKRUI40LHTLZE2VE2YOC4BY13JK1B55


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [48]:
CentralToronto_data.loc[0, 'Neighborhood']

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [49]:
neighborhood_latitude = CentralToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = CentralToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = CentralToronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [51]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OZNSZDTSIOL1JOGR2BOJMLQBSCV1GQEYQFU0C1XWW3DHHAHQ&client_secret=C5DINZ0AKSXVYAIWQUKRUI40LHTLZE2VE2YOC4BY13JK1B55&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

Send the GET request and examine the resutls

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d428b0b5bc9e3002ffc4c81'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [53]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


And how many venues were returned by Foursquare?

In [55]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Central Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Central Toronto*.

In [59]:
# type your answer here

CentralToronto_venues = getNearbyVenues(names=CentralToronto_data['Neighborhood'],
                                   latitudes=CentralToronto_data['Latitude'],
                                   longitudes=CentralToronto_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
North Toronto West
The Annex,North Midtown,Yorkville
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West


#### Let's check the size of the resulting dataframe

In [60]:
print(CentralToronto_venues.shape)
CentralToronto_venues.head()

(116, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Dr.Paul Hodges MIP,43.710634,-79.415810,Health & Beauty Service
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


Let's check how many venues were returned for each neighborhood

In [61]:
CentralToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7
"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",15,15,15,15,15,15
"Forest Hill North,Forest Hill West",5,5,5,5,5,5
Lawrence Park,3,3,3,3,3,3
"Moore Park,Summerhill East",3,3,3,3,3,3
North Toronto West,20,20,20,20,20,20
Roselawn,2,2,2,2,2,2
"The Annex,North Midtown,Yorkville",24,24,24,24,24,24


#### Let's find out how many unique categories can be curated from all the returned venues

In [63]:
print('There are {} uniques categories.'.format(len(CentralToronto_venues['Venue Category'].unique())))

There are 61 uniques categories.


## 3. Analyze Each Neighborhood

In [66]:
# one hot encoding
CentralToronto_onehot = pd.get_dummies(CentralToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CentralToronto_onehot['Neighborhood'] = CentralToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CentralToronto_onehot.columns[-1]] + list(CentralToronto_onehot.columns[:-1])
CentralToronto_onehot = CentralToronto_onehot[fixed_columns]

CentralToronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bookstore,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,...,0,0,0,0,0
1,Lawrence Park,0,0,0,0,...,0,0,0,0,0
2,Lawrence Park,0,0,0,0,...,0,0,0,0,0
3,Roselawn,0,0,0,0,...,0,0,0,0,0
4,Roselawn,0,0,0,0,...,0,0,0,0,0


In [67]:
CentralToronto_onehot.shape

(116, 62)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [68]:
CentralToronto_grouped = CentralToronto_onehot.groupby('Neighborhood').mean().reset_index()
CentralToronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bookstore,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.027027,...,0.027027,0.000000,0.000000,0.000000,0.00
1,Davisville North,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.066667,0.000000,0.066667,0.000000,...,0.000000,0.000000,0.000000,0.066667,0.00
3,"Forest Hill North,Forest Hill West",0.000000,0.000000,0.000000,0.000000,...,0.000000,0.200000,0.000000,0.000000,0.00
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
5,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.00
6,North Toronto West,0.000000,0.000000,0.000000,0.000000,...,0.050000,0.000000,0.000000,0.000000,0.05
7,Roselawn,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00
8,"The Annex,North Midtown,Yorkville",0.041667,0.041667,0.000000,0.000000,...,0.000000,0.000000,0.041667,0.000000,0.00


#### Let's confirm the new size

In [71]:
CentralToronto_grouped.shape

(9, 62)

#### Let's print each neighborhood along with the top 5 most common venues

In [72]:
num_top_venues = 5

for hood in CentralToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CentralToronto_grouped[CentralToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
              venue  freq
0       Pizza Place  0.08
1      Dessert Shop  0.08
2    Sandwich Place  0.08
3              Café  0.05
4  Sushi Restaurant  0.05


----Davisville North----
               venue  freq
0  Food & Drink Shop  0.14
1                Gym  0.14
2     Breakfast Spot  0.14
3               Park  0.14
4              Hotel  0.14


----Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07
3          Supermarket  0.07
4           Restaurant  0.07


----Forest Hill North,Forest Hill West----
              venue  freq
0     Jewelry Store   0.2
1             Trail   0.2
2  Sushi Restaurant   0.2
3          Bus Line   0.2
4              Park   0.2


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1             Bus Line  0.33
2          Swim School  0.33
3  American Restaurant  0.00
4       Sandwich Pla

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CentralToronto_grouped['Neighborhood']

for ind in np.arange(CentralToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CentralToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,...,Café,Coffee Shop,Restaurant,Discount Store,Japanese Restaurant
1,Davisville North,Sandwich Place,Hotel,Gym,Breakfast Spot,...,Clothing Store,Park,Fried Chicken Joint,Discount Store,Farmers Market
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,American Restaurant,Sports Bar,...,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant
3,"Forest Hill North,Forest Hill West",Bus Line,Trail,Jewelry Store,Sushi Restaurant,...,Yoga Studio,Discount Store,Farmers Market,Food & Drink Shop,Fried Chicken Joint
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,...,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [83]:
# set number of clusters
kclusters = 5

CentralToronto_grouped_clustering = CentralToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CentralToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 4, 3, 0, 2, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CentralToronto_merged = CentralToronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CentralToronto_merged = CentralToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CentralToronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,...,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,...,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Fried Chicken Joint
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,...,Clothing Store,Park,Fried Chicken Joint,Discount Store,Farmers Market
3,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,...,Yoga Studio,Discount Store,Farmers Market,Food & Drink Shop,Fried Chicken Joint
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,...,Diner,Mexican Restaurant,Park,Dessert Shop,Cosmetics Shop


Finally, let's visualize the resulting clusters

In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CentralToronto_merged['Latitude'], CentralToronto_merged['Longitude'], CentralToronto_merged['Neighborhood'], CentralToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [88]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 0, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,0,Summer Camp,Trail,Playground,...,Discount Store,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Garden


#### Cluster 2

In [89]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 1, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Health & Beauty Service,Garden,Yoga Studio,...,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Fried Chicken Joint


#### Cluster 3

In [90]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 2, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,2,Sandwich Place,Hotel,Gym,...,Clothing Store,Park,Fried Chicken Joint,Discount Store,Farmers Market
4,Central Toronto,2,Coffee Shop,Sporting Goods Shop,Clothing Store,...,Diner,Mexican Restaurant,Park,Dessert Shop,Cosmetics Shop
5,Central Toronto,2,Sandwich Place,Coffee Shop,Café,...,Park,Pub,Liquor Store,Cosmetics Shop,Convenience Store
6,Central Toronto,2,Pizza Place,Sandwich Place,Dessert Shop,...,Café,Coffee Shop,Restaurant,Discount Store,Japanese Restaurant
8,Central Toronto,2,Pub,Coffee Shop,American Restaurant,...,Fried Chicken Joint,Light Rail Station,Liquor Store,Pizza Place,Restaurant


#### Cluster 4

In [91]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 3, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,3,Swim School,Bus Line,Park,...,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop


#### Cluster 5

In [92]:
CentralToronto_merged.loc[CentralToronto_merged['Cluster Labels'] == 4, CentralToronto_merged.columns[[1] + list(range(5, CentralToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,4,Bus Line,Trail,Jewelry Store,...,Yoga Studio,Discount Store,Farmers Market,Food & Drink Shop,Fried Chicken Joint
